# Function Call

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv()
# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
model_name = "gpt-3.5-turbo-1106"

# シンプルな例

In [2]:
import json

# 検索結果を返す関数の作成
def get_search_result(keyword, when, unit="d"):
    result = {
        "result": [
            {"title": "1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました"},
            {"title": "1929年（昭和4年）八重洲口が開設されました"},
            {"title": "2014年（平成26年）東京駅開業100周年を迎えました"},
        ]
    }

    return json.dumps(result)

In [4]:
from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [5]:
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": "あなたはJSON形式で出力するようデザインされたアシスタントです。"},
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
    response_format={"type":"json_object"}
)

In [6]:
response

ChatCompletion(id='chatcmpl-9GxrPGfv8PnpRb9jl8xa2erMtLqwa', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PHRRqkCsvorw68TWO9O5xmcS', function=Function(arguments='{"keyword":"東京駅のイベント","when":1,"unit":"m"}', name='get_search_result'), type='function')]))], created=1713830379, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_77a673219d', usage=CompletionUsage(completion_tokens=33, prompt_tokens=161, total_tokens=194))

In [7]:
response.choices[0].message.tool_calls[0].function.arguments

'{"keyword":"東京駅のイベント","when":1,"unit":"m"}'

In [8]:
# モデルがツール呼出と判断した
if response.choices[0].message.tool_calls is not None:
    # 言語モデルの回答からツールを取得
    tool = response.choices[0].message.tool_calls[0]

    # 関数名の取得
    function_name = tool.function.name
    print(f"関数名：{function_name}")
    # 引数の取得
    arguments = json.loads(tool.function.arguments)
    print(f"引数：{arguments}")

    # 関数名で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            when=arguments.get("when"),
            unit=arguments.get("unit"),
        )
    print(f"関数の実行結果：{function_response}")

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response2 = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0].message.content.strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print(response.choices[0].message.content.strip())

関数名：get_search_result
引数：{'keyword': '東京駅のイベント', 'when': 1, 'unit': 'm'}
関数の実行結果：{"result": [{"title": "1914\u5e74\uff08\u5927\u6b633\u5e74\uff09\u4e2d\u592e\u505c\u8eca\u5834\u6539\u3081\u300c\u6771\u4eac\u99c5\u300d\u3068\u3057\u3066\u55b6\u696d\u3092\u958b\u59cb\u3057\u307e\u3057\u305f"}, {"title": "1929\u5e74\uff08\u662d\u548c4\u5e74\uff09\u516b\u91cd\u6d32\u53e3\u304c\u958b\u8a2d\u3055\u308c\u307e\u3057\u305f"}, {"title": "2014\u5e74\uff08\u5e73\u621026\u5e74\uff09\u6771\u4eac\u99c5\u958b\u696d100\u5468\u5e74\u3092\u8fce\u3048\u307e\u3057\u305f"}]}
最近1ヶ月以内の東京駅のイベント検索結果は以下の通りです。
1. 1914年(大正3年)中央停車場改め「東京駅」として業務を開始しました
2. 1929年(昭和4年)八重洲口が開設されました
3. 2014年(平成26年)東京駅開業100周年を迎えました
